# Mosquito Bundler

The purpose of this notebook is to allow the user to select a data range and polygon on a map to retrieve all the data from the mosquito bundle protocols for the selection.

Select "Restart & Run All" from the "Kernel" menu and confirm "Restart and Run All Cells."

Open any *** settings -----> sections with the ^ button in the tool bar above.

### Importing Modules and Tools

The following code imports needed modules and subroutines.

In [1]:
# subroutine for designating a code block
def designate(title, section='main'):
    """Designate a code block with a title so that the code may be hidden and reopened.
    
    Arguments:
        title: str, title for code block
        section='main': str, section title
        
    Returns:
        None
    """
    
    # begin designation
    designation = ' ' * 20
    
    # if marked for user parameters
    if section == 'settings':
        
        # begin designation with indicator
        designation = '*** settings -----> '
    
    # add code designator
    designation += '^ [code] (for {}: {})'.format(section, title)
    
    # print
    print(designation)
    
    return None

# apply to itself
designate('designating hidden code blocks', 'designation')

                    ^ [code] (for designation: designating hidden code blocks)


In [2]:
designate('importing Python modules')

# import os and sys modules for system controls
import os
import sys

# import requests and json modules for making API requests
import requests
import json

# set runtime warnings to ignore
import warnings

# import datetime module for manipulating date and time formats
from datetime import datetime

# import iPython for javascript based notebook controls
from IPython.display import Javascript, display, FileLink

# import ipywidgets for additional widgets
from ipywidgets import Button

# import ipyleaflet for the map
from ipyleaflet import Map, DrawControl, basemaps, GeoJSON

# import geopy to get country from coordinates
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderQueryError

# import pandas to create csv files and display tables
import pandas

# import zipfile to create zip files
from zipfile import ZipFile

                    ^ [code] (for main: importing Python modules)


In [3]:
designate('defining global variables')

# disable runtime warnings
warnings.filterwarnings('ignore')

# set pandas optinos
pandas.set_option("display.max_rows", None)
pandas.set_option("display.max_columns", None)

# default link
link = None

                    ^ [code] (for main: defining global variables)


In [4]:
designate('scanning notebook for cells', 'introspection')

# scan notebook for cell information
def scan():
    """Scan the notebook and collect cell information.

    Arguments:
        None

    Returns:
        list of dicts
    """

    # open the notebook file 
    with open('bundler.ipynb', 'r', encoding='utf-8') as pointer:
        
        # and read its contents
        contents = json.loads(pointer.read())

    # get all cells
    cells = contents['cells']

    return cells

                    ^ [code] (for introspection: scanning notebook for cells)


In [5]:
designate('looking for particular cell', 'navigation')

# function to look for cells with a particular text snippet
def look(text):
    """Look for a particular text amongst the cells.
    
    Arguments:
        text: str, the text to search for
        
    Returns:
        list of int, the cell indices.
    """
    
    # get cells
    cells = scan()
    
    # search for cells 
    indices = []
    for index, cell in enumerate(cells):
        
        # search for text in source
        if any([text in line for line in cell['source']]):
            
            # but disregard if text is in quotes
            if all(["'{}'".format(text) not in line for line in cell['source']]):
            
                # add to list
                indices.append(index)
            
    return indices

                    ^ [code] (for navigation: looking for particular cell)


In [6]:
designate('executing cell range by text', 'navigation')

# execute cell range command
def execute(start, finish):
    """Execute a cell range based on text snippets.
    
    Arguments:
        start: str, text from beginning cell of range
        finish: str, text from ending cell of range
        
    Returns:
        None
    """
    
    # find start and finish indices, adding 1 to be inclusive
    opening = look(start)[0] 
    closing = look(finish)[0]
    bracket = (opening, closing)
    
    # make command
    command = 'IPython.notebook.execute_cell_range' + str(bracket)
    
    # perform execution
    display(Javascript(command))
    
    return None

                    ^ [code] (for navigation: executing cell range by text)


In [7]:
designate('refreshing cells by relative position', 'navigation')

# execute cell range command
def refresh(start, finish=None):
    """Refresh a particular cell relative to current cell.
    
    Arguments:
        start: int, the first cell offset
        finish=None: int, the second cell offset
        
    Returns:
        None
    """
    
    # make offset into a string
    stringify = lambda offset: str(offset) if offset < 0 else '+' + str(offset)
    
    # default finish to start
    finish = finish or start
    
    # make command
    command = 'IPython.notebook.execute_cell_range('
    command += 'IPython.notebook.get_selected_index()' + stringify(start) + ','
    command += 'IPython.notebook.get_selected_index()' + stringify(finish + 1) + ')'
    
    # perform execution
    display(Javascript(command))
    
    return None

                    ^ [code] (for navigation: refreshing cells by relative position)


In [8]:
designate('ridding polygons', 'mapping')

# function to clear old polygons with drawing of newone
def rid(self, action, geo_json):
    """Rid the map of previous polygons, keeping only the one drawn.
    
    Arguments:
        self: self
        action: action
        geo_json: dict
        
    Returns:
        None
    """
    
    # clear polygons and rectanges from draw control
    chart.controls[-1].clear_polygons()
    chart.controls[-1].clear_rectangles()
    
    # remove all previous layers
    chart.layers = chart.layers[:1]
    
    # add polygon to chart
    chart.add_layer(GeoJSON(data=geo_json))
        
    return None

                    ^ [code] (for mapping: ridding polygons)


In [9]:
designate('forcing rectanges', 'mapping')

# force a polygon on the map
def force(south, north, west, east):
    """Force a rectangle onto the map.
    
    Arguments:
        south: float
        north: float
        west: float
        east: float
        
    Returns:
        None
    """
    
    # check for values
    if all([cardinal is not None for cardinal in [south, north, west, east]]):
        
        # construct coordinates
        coordinates = [[[west, south], [west, north], [east, north], [east, south], [west, south]]]
    
        # construct geo_json
        geo_json = {'type': 'Feature'}
        geo_json['properties'] = {'style': chart.controls[-1].rectangle['shapeOptions']}
        geo_json['geometry'] = {'type': 'Polygon', 'coordinates': coordinates}
                
        # add rectangle to chart
        chart.add_layer(GeoJSON(data=geo_json))
    
    return None

                    ^ [code] (for mapping: forcing rectanges)


In [10]:
designate('locating country from geocoordinates', 'mapping')

# locate function
def locate(longitude, latitude):
    """Locate the country from a geocoordinate pair:
    
    Arguments:
        latitude: float
        longitude: float
        
    Returns:
        str
    """

    # try to get country
    country = ''
    try:
        
        # get country name
        locator = Nominatim(user_agent="GLOBE")
        query = ', '.join([str(latitude), str(longitude)])
        location = locator.reverse(query, language='english')
        country = location.raw['address']['country']
        country = '{}'.format(country.encode('ascii', errors='ignore').decode())
        
    # otherwise
    except (KeyError, GeocoderQueryError):
        
        # pass
        pass
    
    return country

                    ^ [code] (for mapping: locating country from geocoordinates)


In [11]:
designate('retrieving the surrounding polygon', 'mapping')

# getting polygon from map routine
def surround(chart):
    """Get the polygon surrounding the area on the map
    
    Arguments:
        chart: ipyleaflet chart
        
    Returns:
        list of points, polygon
    """

    # try to retrieve the polygon
    try:

        # get polygon
        polygon = chart.layers[1].data['geometry']['coordinates'][0]

    # unless it is not available
    except IndexError:
        
        # set to message
        polygon = ['no polygon drawn yet, please draw one on the map']
        
    return polygon

                    ^ [code] (for mapping: retrieving the surrounding polygon)


In [12]:
designate('record flattening', 'processing')

# function to flatten a nested list into a single-level structure
def flatten(record, label=None):
    """Flatten each record into a single level.

    Arguments:
        record: dict, a record
        label: str, key from last nesting

    Returns:
        dict
    """

    # initiate dictionary
    flattened = {}

    # try to flatten the record
    try:

        # go through each field
        for field, info in record.items():

            # and flatten the smaller records found there
            flattened.update(flatten(info, field))

    # otherwise record is a terminal entry
    except AttributeError:

        # so update the dictionary with the record
        flattened.update({label: record})

    return flattened

                    ^ [code] (for processing: record flattening)


In [13]:
designate('calling the api subroutine', 'api')

# call the api with protocol and country code
def call(protocol, beginning, ending, polygon):
    """Call the api:
    
    Arguments:
        protocol: str, the protocol
        beginning: str, the beginning date
        ending: str, the ending date
        polygon: list of points
        
    Returns:
        list of dicts, the records
    """
    
    # assemble the url for the API call 
    url = 'https://api.globe.gov/search/v1/measurement/protocol/measureddate/polygon/geojson/'
    url += '?protocols=' + protocol
    url += '&startdate=' + beginning 
    url += '&enddate=' + ending
    
    # begin with first point, and continue
    coordinates = '%5B%5B' + str(polygon[0][0]) + '%2C%20' + str(polygon[0][1])
    for point in polygon[1:]:
    
        # add points
        coordinates += '%5D%2C%20%5B' + str(point[0]) + '%2C%20' + str(point[1])
        
    # end with cap
    coordinates += '%5D%5D'
    url += '&coordinates=' + coordinates

    # geojson parameter toggles between formats
    url += '&geojson=FALSE'
    
    # sample parameter returns small sample set if true
    url += '&sample=FALSE'
    
    # make the API call and return the raw results
    request = requests.get(url)
    raw = json.loads(request.text)
    
    return raw

                    ^ [code] (for api: calling the api subroutine)


In [14]:
designate('collecting results from all protocols', 'api')

# collecting results
def collect(polygon):
    """Collect all data from within dates and polygon on map.
    
    Arguments:
        polygon: list of points
        
    Returns:
        list of panda frames
    """
    
    # check polygon
    assert polygon[0] != str(polygon[0]), 'no polygon drawn on map yet'
    
    # set protocol list
    mosquitoes = 'mosquito_habitat_mapper'
    protocols = [mosquitoes] + secondaries
    
    # begin zipfile
    date = str(datetime.now().date()).replace('-', '')
    now = str(int(datetime.now().timestamp()))
    bundle = 'mosquitoes_bundle_' + date + '_' + now +'.zip'
    album = ZipFile(bundle, 'w')

    # collect results
    for protocol in protocols:

        # make call
        print('\nmaking request from {}...'.format(protocol))
        raw = call(protocol, beginning, ending, polygon)
        result = raw['results']
        length = len(result)
        message = '{} results returned from {}.\n'.format(length, protocol)
        summary.append(message)
        print(message)

        # flatten all records
        records = [flatten(record) for record in result]
        panda = pandas.DataFrame(records)
        
        # write dataframe to file
        name = protocol + '_' + date + '_' + now + '.csv'
        panda.to_csv(name)  
        album.write(name)
        
        # delete file
        os.remove(name)
        
        # display sample
        display(panda.head(5))
        
    # create summary
    path = 'summary_' + date + '_' + now + '.txt'
    with open(path, 'w') as pointer:
        
        # write summary file
        pointer.writelines(summary)
        
    # add to album and remove from main directory
    album.write(path)
    os.remove(path)
        
    # make link
    link = FileLink(bundle)

    return link

                    ^ [code] (for api: collecting results from all protocols)


In [108]:
designate('import status')

# print status
print('modules imported.')

                    ^ [code] (for main: import status)
modules imported.


### MyBinder Link

This notebook is available at the following link hosted by MyBinder:

https://mybinder.org/v2/git/https%3A%2F%2Fmattbandel%40bitbucket.org%2Fmattbandel%2Fglobe-mosquitoes-bundler.git/master?filepath=bundler.ipynb

### Mosquito Bundle Protocols

Data from the following GLOBE protocols are considered part of the mosquitoes bundle.

In [16]:
designate('table of mosquito bundle protocols')

# get from secondaries file
with open('protocols.txt', 'r') as pointer:
    
    # get all secondaries
    secondaries = [protocol.strip('\n') for protocol in pointer.readlines()]
    secondaries = [protocol for protocol in secondaries if 'X' in protocol]
    secondaries = [protocol.strip('X').strip() for protocol in secondaries]

# print as list
print('{} mosquito bundle protocols:'.format(len(secondaries)))
secondaries

                    ^ [code] (for main: table of mosquito bundle protocols)
21 mosquito bundle protocols:


['air_temp_dailies',
 'air_temp_monthlies',
 'air_temp_noons',
 'air_temps',
 'conductivities',
 'dissolved_oxygens',
 'freshwater_macroinvertebrates',
 'humidities',
 'humidity_monthlies',
 'humidity_noons',
 'hydrology_alkalinities',
 'hydrology_phs',
 'land_covers',
 'precipitation_monthlies',
 'precipitations',
 'salinities',
 'surface_temperature_noons',
 'surface_temperatures',
 'transparencies',
 'vegatation_covers',
 'water_temperatures']

### Setting the Geographic Area

Set latitude and longitude boundaries for the study area and click Apply.

In [111]:
designate('setting the latitude and longitude', 'settings')

# set the latitude and longitude boundaries
south = None
north = None
west = None
east = None

*** settings -----> ^ [code] (for settings: setting the latitude and longitude)


In [112]:
designate('button to apply rectangle')

# function to retrieve all data
def retrieve(_):
    """Retrieve the data from the api.
    
    Arguments:
        None
        
    Returns:
        None
    """

    # refresh cells
    execute('### Setting the Geographic Area', '### Setting the Date Range')
    
    return None

# create button
button = Button(description="Apply")
button.on_click(retrieve)
display(button)

<IPython.core.display.Javascript object>

                    ^ [code] (for main: button to apply rectangle)


Button(description='Apply', style=ButtonStyle())

Or use the pentagon tool or rectangle tool to draw a polygon on the map.

In [113]:
designate('constructing map')

# print status
print('constructing map...')

# set up map with topographical basemap
chart = Map(basemap=basemaps.Esri.WorldTopoMap, center=(0, 0), zoom=3)

# initiate draw control
control = DrawControl()

# specify polygon
control.polygon = {'shapeOptions': {'fillColor': '#00ff00', 'color': '#ffffff', 'fillOpacity': 0.1}}
control.polygon['shapeOptions'].update({'color': '#ffffff', 'weight': 4, 'opacity': 0.5, 'stroke': True})
control.polygon['drawError'] = {'color': '#dd253b', 'message': 'Oops!'}
control.polygon['allowIntersection'] = False

# specify rectange marker
control.rectangle = {'shapeOptions': {'fillColor': '#00ff00', 'color': '#ffffff', 'fillOpacity': 0.1}}
control.rectangle['shapeOptions'].update({'color': '#ffffff', 'weight': 4, 'opacity': 0.5, 'stroke': True})

# remove default polyline and circlemarker
control.polyline = {}
control.circlemarker = {}

# specify clear function
control.on_draw(rid)

# add draw control
chart.add_control(control)

# force a rectange onto the map
force(south, north, west, east)

# display chart
chart

                    ^ [code] (for main: constructing map)
constructing map...


Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

### Setting the Date Range

Set the date range below.  Leaving the beginning date blank will default to Earth Day 1995.  Leaving the ending date blank will default to today's date.

In [103]:
designate('setting date range', 'settings')

# set beginning and ending dates in 'YYYY-mm-dd' format, None for ending date defaults to now
beginning = '2019-01-01'
ending = '2020-01-01'

*** settings -----> ^ [code] (for settings: setting date range)


### Retrieving the Data

Press the Retrieve button believe to retrieve the data.  A link will appear to a zip file below.

In [104]:
designate('resolving default dates')

# default beginning to 1995 and ending to current date if unspecified
beginning = beginning or '1995-04-22'
ending = ending or str(datetime.now().date())

# begin summary
today = datetime.now().date()
clock = datetime.now().time().replace(microsecond=0)
message = 'Summary of Mosquito Bundle Query at {} on {}:\n'.format(clock, today)
summary = [message]
print(message)

# print date range
message = 'date range: {} to {}\n'.format(beginning, ending)
summary.append(message)
print(message)

# retrieve polygon from map
message = 'polygon:\n'
polygon = surround(chart)
summary.append(message)
print(message)
for pair in polygon:

    # print and add to summary
    country = locate(pair[0], pair[1])
    message = '{}: {}'.format(pair, country)
    summary.append(message + '\n')
    print(message)

                    ^ [code] (for main: resolving default dates)
Summary of Mosquito Bundle Query at 18:07:19 on 2020-06-18:

date range: 2019-01-01 to 2020-01-01

polygon:

[-20, 34]: 
[-20, 59]: 
[41, 59]: 
[41, 34]: Iraq
[-20, 34]: 


In [105]:
designate('button to retrieve data')

# function to retrieve all data
def retrieve(_):
    """Retrieve the data from the api.
    
    Arguments:
        None
        
    Returns:
        None
    """

    # refresh cells
    execute('### Setting the Date Range', '### Thanks!')
    
    return None

# create button
button = Button(description="Retrieve")
button.on_click(retrieve)
display(button)

<IPython.core.display.Javascript object>

                    ^ [code] (for main: button to retrieve data)


Button(description='Retrieve', style=ButtonStyle())

In [106]:
designate('collecting data')

# collect data
link = collect(polygon)

                    ^ [code] (for main: collecting data)

making request from mosquito_habitat_mapper...
74 results returned from mosquito_habitat_mapper.



,protocol,measuredDate,createDate,updateDate,publishDate,organizationId,organizationName,siteId,siteName,countryName,countryCode,latitude,longitude,elevation,pid,mosquitohabitatmapperBreedingGroundEliminated,mosquitohabitatmapperExtraData,mosquitohabitatmapperAbdomenCloseupPhotoUrls,mosquitohabitatmapperMeasuredAt,mosquitohabitatmapperMeasurementElevation,mosquitohabitatmapperUserid,mosquitohabitatmapperLarvaeCount,mosquitohabitatmapperMosquitoEggs,mosquitohabitatmapperMosquitoEggCount,mosquitohabitatmapperGenus,mosquitohabitatmapperWaterSource,mosquitohabitatmapperMosquitoAdults,mosquitohabitatmapperSpecies,mosquitohabitatmapperComments,mosquitohabitatmapperMosquitoPupae,mosquitohabitatmapperWaterSourcePhotoUrls,mosquitohabitatmapperDataSource,mosquitohabitatmapperLarvaFullBodyPhotoUrls,mosquitohabitatmapperMeasurementLatitude,mosquitohabitatmapperLastIdentifyStage,mosquitohabitatmapperWaterSourceType,mosquitohabitatmapperMosquitoHabitatMapperId,mosquitohabitatmapperMeasurementLongitude
0,mosquito_habitat_mapper,2019-04-08,2020-01-25T18:24:27,2020-01-25T18:24:27,2020-03-20T22:19:48,31759627,Osnovna škola Rugvica,104783,33TWL957671,Croatia,HRV,45.750833,16.230409,101.0,159355552,True,None,None,2019-04-08T15:10:00,0,31759148,0,False,None,None,dish or pot,True,None,None,False,https://data.globe.gov/system/photos/2019/04/0...,GLOBE Observer App,None,45.7512,None,container: artificial,10501,16.2314
1,mosquito_habitat_mapper,2019-04-16,2020-01-25T18:24:27,2020-01-25T18:24:27,2020-03-20T22:19:48,17769359,Croatia Citizen Science,107698,33TWL430385,None,None,45.498706,15.550375,111.5,159355969,True,None,None,2019-04-16T11:24:00,0,49124038,None,None,None,None,tire,None,None,None,False,None,GLOBE Observer App,None,45.4990,identify,container: artificial,11719,15.5516
2,mosquito_habitat_mapper,2019-04-16,2020-01-25T18:24:27,2020-01-25T18:24:27,2020-03-20T22:19:48,17769359,Croatia Citizen Science,107698,33TWL430385,None,None,45.498706,15.550375,111.5,159355969,False,None,None,2019-04-16T11:32:00,0,49124038,None,None,None,None,tire,None,None,None,False,None,GLOBE Observer App,None,45.4992,identify,container: artificial,11720,15.5515
3,mosquito_habitat_mapper,2019-02-28,2020-01-25T18:16:36,2020-01-25T18:16:36,2020-03-20T22:19:48,107657,OS prof. Franje Viktora Signjara,112554,33TXM542034,Croatia,HRV,46.066782,16.993831,132.6,159354429,False,None,None,2019-02-28T13:32:00,0,3080584,None,None,None,None,swamp or wetland,None,None,None,False,None,GLOBE Observer App,None,46.0671,identify,still: lake/pond/swamp,8048,16.9951
4,mosquito_habitat_mapper,2019-04-11,2020-01-25T18:24:27,2020-01-25T18:24:27,2020-03-20T22:19:48,17769359,Croatia Citizen Science,130796,33TXH769545,None,None,42.922608,17.167573,17.6,159355645,True,None,None,2019-04-11T17:29:00,0,54462226,None,None,None,None,"cement, metal or plastic tank",None,None,None,False,None,GLOBE Observer App,None,42.9233,identify-later,container: artificial,10855,17.1687



making request from air_temp_dailies...
32974 results returned from air_temp_dailies.



,protocol,measuredDate,createDate,updateDate,publishDate,organizationId,organizationName,siteId,siteName,countryName,countryCode,latitude,longitude,elevation,pid,airtempdailiesSolarMeasuredAt,airtempdailiesSolarNoonAt,airtempdailiesCurrentTemp,airtempdailiesMeasuredAt,airtempdailiesMaximumTemp,airtempdailiesMinimumTemp,airtempdailiesComments,airtempdailiesUserid
0,air_temp_dailies,2019-01-01,2020-01-23T16:04:39,2020-01-23T16:04:39,2020-03-20T21:00:00,61017,Bundeshandelsakademie und Bundeshandelsschule ...,208,School Location:ATM-01,Austria,AUT,47.49139,9.72331,403.0,149506245,2019-01-01T11:35:00,2019-01-01T11:24:00,3.2,2019-01-01T11:00:00,4.8,2.9,None,2971264.0
1,air_temp_dailies,2019-01-02,2020-01-23T16:04:39,2020-01-23T16:04:39,2020-03-20T21:00:00,61017,Bundeshandelsakademie und Bundeshandelsschule ...,208,School Location:ATM-01,Austria,AUT,47.49139,9.72331,403.0,149506246,2019-01-02T11:34:00,2019-01-02T11:25:00,2.4,2019-01-02T11:00:00,5.4,1.3,None,2971264.0
2,air_temp_dailies,2019-01-03,2020-01-23T16:04:39,2020-01-23T16:04:39,2020-03-20T21:00:00,61017,Bundeshandelsakademie und Bundeshandelsschule ...,208,School Location:ATM-01,Austria,AUT,47.49139,9.72331,403.0,149506247,2019-01-03T11:34:00,2019-01-03T11:25:00,-0.3,2019-01-03T11:00:00,2.8,-2.5,None,2971264.0
3,air_temp_dailies,2019-01-04,2020-01-23T16:04:39,2020-01-23T16:04:39,2020-03-20T21:00:00,61017,Bundeshandelsakademie und Bundeshandelsschule ...,208,School Location:ATM-01,Austria,AUT,47.49139,9.72331,403.0,149506248,2019-01-04T11:33:00,2019-01-04T11:25:00,0.4,2019-01-04T11:00:00,0.5,-2.0,None,2971264.0
4,air_temp_dailies,2019-01-05,2020-01-23T16:04:39,2020-01-23T16:04:39,2020-03-20T21:00:00,61017,Bundeshandelsakademie und Bundeshandelsschule ...,208,School Location:ATM-01,Austria,AUT,47.49139,9.72331,403.0,149506249,2019-01-05T11:33:00,2019-01-05T11:26:00,-0.3,2019-01-05T11:00:00,1.4,-0.3,None,2971264.0



making request from air_temp_monthlies...
1074 results returned from air_temp_monthlies.



...

,protocol,measuredDate,createDate,updateDate,publishDate,organizationId,organizationName,siteId,siteName,countryName,countryCode,latitude,longitude,elevation,pid,transparenciesComments,transparenciesWaterBodyState,transparenciesTransparencyDiskDoesNotDisappear,transparenciesUserid,transparenciesMeasuredAt,transparenciesTransparencyDiskImageDisappearanceM,transparenciesTubeImageDisappearanceCm,transparenciesTubeImageDoesNotDisappear
0,transparencies,2019-01-06,2020-01-23T02:46:00,2020-01-23T02:46:00,2020-04-25T21:34:07,103048,ZS Bystrice nad Pernstejnem,1960,School Location:SWS-01,Czech Republic,CZE,49.519,16.26,570.0,139487377,None,normal,False,3081588,2019-01-06T08:00:00,NaN,31.0,False
1,transparencies,2019-01-13,2020-01-23T02:46:00,2020-01-23T02:46:00,2020-04-25T21:34:07,103048,ZS Bystrice nad Pernstejnem,1960,School Location:SWS-01,Czech Republic,CZE,49.519,16.26,570.0,139487379,None,normal,False,3081588,2019-01-13T08:00:00,NaN,53.0,False
2,transparencies,2019-01-21,2020-01-23T02:46:00,2020-01-23T02:46:00,2020-04-25T21:34:07,103048,ZS Bystrice nad Pernstejnem,1960,School Location:SWS-01,Czech Republic,CZE,49.519,16.26,570.0,139487378,None,normal,False,3081588,2019-01-21T08:00:00,NaN,40.0,False
3,transparencies,2019-01-27,2020-01-23T02:46:00,2020-01-23T02:46:00,2020-04-25T21:34:07,103048,ZS Bystrice nad Pernstejnem,1960,School Location:SWS-01,Czech Republic,CZE,49.519,16.26,570.0,139487380,None,normal,False,3081588,2019-01-27T08:00:00,NaN,120.0,True
4,transparencies,2019-02-03,2020-01-23T02:46:01,2020-01-23T02:46:01,2020-04-25T21:34:07,103048,ZS Bystrice nad Pernstejnem,1960,School Location:SWS-01,Czech Republic,CZE,49.519,16.26,570.0,139488649,None,normal,False,3081588,2019-02-03T08:00:00,NaN,15.0,False



making request from vegatation_covers...
2290 results returned from vegatation_covers.



,protocol,measuredDate,createDate,updateDate,publishDate,organizationId,organizationName,siteId,siteName,countryName,countryCode,latitude,longitude,elevation,pid,vegatationcoversCanopyCoverEvergreenPercent,vegatationcoversDwarfShrubCoverMinusCount,vegatationcoversCanopyCoverShrubCount,vegatationcoversUserid,vegatationcoversCanopyCoverObservationsCount,vegatationcoversDwarfShrubCoverPlusCount,vegatationcoversDwarfShrubCoverPlusPercent,vegatationcoversGroundCoverBrownCount,vegatationcoversCanopyCoverEvergreenCount,vegatationcoversCanopyCoverPlusPercent,vegatationcoversGroundCoverOtherPercent,vegatationcoversGroundCoverGreenPercent,vegatationcoversCanopyCoverShrubPercent,vegatationcoversGroundCoverShrubPercent,vegatationcoversGroundCoverObservationsCount,vegatationcoversGroundCoverGraminoidCount,vegatationcoversGroundCoverDwarfShrubCount,vegatationcoversCanopyCoverPlusCount,vegatationcoversShrubCoverPlusPercent,vegatationcoversShrubCoverMinusCount,vegatationcoversGroundCoverOtherCount,vegatationcoversGroundCoverForbPercent,vegatationcoversDwarfShrubCoverObservationsCount,vegatationcoversMeasuredOn,vegatationcoversGroundCoverGreenCount,vegatationcoversCanopyCoverTreeCount,vegatationcoversCanopyCoverDeciduousCount,vegatationcoversCanopyCoverDeciduousPercent,vegatationcoversShrubCoverPlusCount,vegatationcoversGroundCoverBrownPercent,vegatationcoversShrubCoverObservationsCount,vegatationcoversComments,vegatationcoversGroundCoverMinusCount,vegatationcoversCanopyCoverMinusCount,vegatationcoversGroundCoverShrubCount,vegatationcoversGroundCoverDwarfShrubPercent,vegatationcoversGroundCoverPlusCount,vegatationcoversGroundCoverForbCount,vegatationcoversGroundCoverPlusPercent,vegatationcoversCanopyCoverTreePercent,vegatationcoversGroundCoverGraminoidPercent
0,vegatation_covers,2019-04-01,2020-01-23T02:47:29,2020-01-23T02:47:29,2020-06-15T21:18:08,104203,IES Esteban Manuel de Villegas,3886,ERA CAZUELA:BIO-02,Spain,ESP,42.355000,-2.655200,780.0,139502796,0.0,19.0,0.0,3129296,20.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,12.0,1.0,14.0,0.0,19.0,0.0,0.0,20.0,2019-04-01T00:00:00,14.0,14.0,14.0,100.0,1.0,0.0,20.0,None,6.0,6.0,1.0,0.0,14.0,0.0,0.0,0.0,0.0
1,vegatation_covers,2019-02-01,2020-01-23T02:47:27,2020-01-23T02:47:27,2020-06-15T21:18:08,236592,OS Valentin Klarin,5031,HRAST CRNIKA,Croatia,HRV,44.082000,15.178600,31.0,139500996,100.0,40.0,0.0,3074060,40.0,0.0,0.0,2.0,33.0,0.0,0.0,0.0,0.0,0.0,40.0,2.0,0.0,33.0,0.0,40.0,2.0,0.0,40.0,2019-02-01T00:00:00,2.0,33.0,0.0,0.0,0.0,0.0,40.0,None,36.0,7.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
2,vegatation_covers,2019-10-11,2020-01-23T02:47:35,2020-01-23T02:47:35,2020-06-15T21:18:08,236592,OS Valentin Klarin,5031,HRAST CRNIKA,Croatia,HRV,44.082000,15.178600,31.0,139506391,100.0,40.0,0.0,3074060,40.0,0.0,0.0,2.0,33.0,0.0,0.0,0.0,0.0,0.0,40.0,2.0,0.0,33.0,0.0,40.0,2.0,0.0,40.0,2019-10-11T00:00:00,2.0,33.0,0.0,0.0,0.0,0.0,40.0,None,36.0,7.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
3,vegatation_covers,2019-04-08,2020-01-23T02:47:29,2020-01-23T02:47:29,2020-06-15T21:18:08,143492,OS Hugo Badalic,5100,BMjerenja:BIO-01,Croatia,HRV,45.187333,18.000309,155.5,139502797,NaN,0.0,0.0,11953661,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,2019-04-08T00:00:00,0.0,0.0,0.0,NaN,0.0,NaN,0.0,None,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN
4,vegatation_covers,2019-04-18,2020-01-23T02:47:29,2020-01-23T02:47:29,2020-06-15T21:18:08,143492,OS Hugo Badalic,5100,BMjerenja:BIO-01,Croatia,HRV,45.187333,18.000309,155.5,139502798,NaN,0.0,0.0,11953661,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,2019-04-18T00:00:00,0.0,0.0,0.0,NaN,0.0,NaN,0.0,None,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN



making request from water_temperatures...
3119 results returned from water_temperatures.



,protocol,measuredDate,createDate,updateDate,publishDate,organizationId,organizationName,siteId,siteName,countryName,countryCode,latitude,longitude,elevation,pid,watertemperaturesThermometerProbeMfg,watertemperaturesWaterBodyState,watertemperaturesThermometerProbeModel,watertemperaturesComments,watertemperaturesWaterTempC,watertemperaturesTemperatureMethod,watertemperaturesUserid,watertemperaturesMeasuredAt
0,water_temperatures,2019-01-06,2020-01-23T04:41:22,2020-01-23T04:41:22,2020-04-25T21:34:26,103048,ZS Bystrice nad Pernstejnem,1960,School Location:SWS-01,Czech Republic,CZE,49.519,16.26,570.0,141312498,None,normal,None,None,1.0,probe,3081588,2019-01-06T08:00:00
1,water_temperatures,2019-01-13,2020-01-23T04:41:22,2020-01-23T04:41:22,2020-04-25T21:34:26,103048,ZS Bystrice nad Pernstejnem,1960,School Location:SWS-01,Czech Republic,CZE,49.519,16.26,570.0,141312501,None,normal,None,None,2.0,probe,3081588,2019-01-13T08:00:00
2,water_temperatures,2019-01-21,2020-01-23T04:41:22,2020-01-23T04:41:22,2020-04-25T21:34:26,103048,ZS Bystrice nad Pernstejnem,1960,School Location:SWS-01,Czech Republic,CZE,49.519,16.26,570.0,141312500,None,normal,None,None,0.0,probe,3081588,2019-01-21T08:00:00
3,water_temperatures,2019-01-27,2020-01-23T04:41:22,2020-01-23T04:41:22,2020-04-25T21:34:26,103048,ZS Bystrice nad Pernstejnem,1960,School Location:SWS-01,Czech Republic,CZE,49.519,16.26,570.0,141312499,None,normal,None,None,0.0,probe,3081588,2019-01-27T08:00:00
4,water_temperatures,2019-02-03,2020-01-23T04:41:24,2020-01-23T04:41:24,2020-04-25T21:34:26,103048,ZS Bystrice nad Pernstejnem,1960,School Location:SWS-01,Czech Republic,CZE,49.519,16.26,570.0,141314163,None,normal,None,None,2.0,alcohol-filled thermometer,3081588,2019-02-03T08:00:00


In [107]:
designate('displaying link')

# attemmpt to display link
if link:
    
    # display last link
    display(link)

                    ^ [code] (for main: displaying link)


/Users/matthewbandel/PycharmProjects/globe-mosquitoes-bundler/mosquitoes_bundle_20200618_1592528842.zip

### Thanks!

Please feel free to direct questions or feedback to Matthew Bandel at matthew.bandel@ssaihq.com